<a href="https://colab.research.google.com/github/beatuphat/for_fun/blob/main/Volleyball_Analytics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# prompt: load a google sheet

from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default

# Get credentials and project ID.
creds, _ = default()

# Authorize the client.
gc = gspread.authorize(creds)

# Open the Google Sheet by its URL.
sheet = gc.open_by_url('https://docs.google.com/spreadsheets/d/1nuBzxaLVbTkQoN61a3WZ1YmVV9npdDdZQj0USXJiM0c/edit?usp=sharing')

# Select the first worksheet.
# worksheet = sheet.get_worksheet(0)

# Select a worksheet by title.
worksheet = sheet.worksheet("results")

# Get all values from the sheet.
data = worksheet.get_all_values()

# Print the data.
print(data)


In [ ]:
# prompt: create a dataframe from the worksheet

import pandas as pd

# Create a Pandas DataFrame from the data.
df = pd.DataFrame(data[1:], columns=data[0])

# Print the DataFrame.
print(df)


In [ ]:
# prompt: profile the data types

df.info()


In [ ]:
# Create a list to store the player columns
player_columns = [col for col in df.columns if 'player' in col.lower()]
player_columns

In [ ]:
# Create a list to store the players
df_players = []

# Add players from each team to the list
df_players.extend(df['team 1 player 1'])
df_players.extend(df['team 1 player 2'])
df_players.extend(df['team 2 player 1'])
df_players.extend(df['team 2 player 2'])

# Convert the list to a Series
df_players = pd.Series(df_players)

df_players

In [ ]:
# prompt: get the distinct values from df_players

# Get the distinct values from df_players
distinct_players = df_players.unique()
print(distinct_players)


In [ ]:
# prompt: generate win loss results for each player using the columns winner #1, winner #2, loser #1, loser #2

# Create a dictionary to store the win-loss results for each player
player_results = {}

# Initialize the dictionary with players and their initial win-loss counts
for player in distinct_players:
  if player:  # Check if the player name is not empty
    player_results[player] = {'wins': 0, 'losses': 0}

# Iterate through the DataFrame
for index, row in df.iterrows():
  # Get the winners and losers
  winners = [row['winner #1'], row['winner #2']]
  losers = [row['loser #1'], row['loser #2']]

  # Update the win-loss counts for winners
  for winner in winners:
    if winner in player_results:
      player_results[winner]['wins'] += 1

  # Update the win-loss counts for losers
  for loser in losers:
    if loser in player_results:
      player_results[loser]['losses'] += 1

# Print the win-loss results for each player
for player, result in player_results.items():
  print(f"{player}: Wins - {result['wins']}, Losses - {result['losses']}")


In [ ]:
player_results

In [ ]:
# prompt: make player_results into a dataframe

import pandas as pd
# Create a DataFrame from the player_results dictionary
df_player_results = pd.DataFrame.from_dict(player_results, orient='index')

# Reset the index to make the player names a column
df_player_results = df_player_results.reset_index()

# Rename the columns
df_player_results = df_player_results.rename(columns={'index': 'Player', 'wins': 'Wins', 'losses': 'Losses'})
df_player_results['Total Games'] = df_player_results['Wins'] + df_player_results['Losses']
df_player_results['Win Percentage'] = (df_player_results['Wins'] / df_player_results['Total Games']) * 100

# Print the DataFrame
print(df_player_results)


In [ ]:
# prompt: order df_player_results by win percentage

# Sort the DataFrame by Win Percentage in descending order
df_player_results = df_player_results.sort_values(by=['Win Percentage'], ascending=False)

# Print the sorted DataFrame
print(df_player_results)


In [ ]:
# prompt: calculate the TruVolley ratings

import math

# Function to calculate TruVolley rating
def calculate_truvolley_rating(wins, losses):
  """
  Calculates the TruVolley rating based on wins and losses.

  Args:
    wins: The number of wins.
    losses: The number of losses.

  Returns:
    The TruVolley rating.
  """
  if wins + losses == 0:
    return 0

  win_ratio = wins / (wins + losses)
  rating = math.log10(wins + losses) * (2 * win_ratio - 1)
  return rating

# Add a new column for TruVolley rating
df_player_results['TruVolley Rating'] = df_player_results.apply(lambda row: calculate_truvolley_rating(row['Wins'], row['Losses']), axis=1)

# Sort the DataFrame by TruVolley Rating in descending order
df_player_results = df_player_results.sort_values(by=['TruVolley Rating'], ascending=False)

# Print the DataFrame with TruVolley ratings
print(df_player_results)


In [ ]:
# https://github.com/tjbreshears/VBelo/blob/main/vbelo.py
# Function to calculate the Probability of winning/losing
def probability(rating1, rating2):
    return 1.0 * 1.0 / (1 + 1.0 * math.pow(10, 1.0 * (rating1 - rating2) / 400))

In [ ]:
# https://github.com/openvolley/pydatavolley

In [ ]:
import pandas as pd
# Function to calculate ELO rating
def calculate_elo_rating(rating_a, rating_b, score_a, score_b, k_factor=32):
  """
  Calculates the new ELO ratings for two players after a game.

  Args:
    rating_a: The current ELO rating of player A.
    rating_b: The current ELO rating of player B.
    score_a: The score of player A.
    score_b: The score of player B.
    k_factor: The K-factor used in the ELO calculation (default is 32).

  Returns:
    A tuple containing the new ELO ratings for player A and player B.
  """
  expected_score_a = 1 / (1 + 10 ** ((rating_b - rating_a) / 400))
  expected_score_b = 1 / (1 + 10 ** ((rating_a - rating_b) / 400))

  if score_a > score_b:
    actual_score_a = 1
    actual_score_b = 0
  elif score_a < score_b:
    actual_score_a = 0
    actual_score_b = 1
  else:
    actual_score_a = 0.5
    actual_score_b = 0.5

  new_rating_a = rating_a + k_factor * (actual_score_a - expected_score_a)
  new_rating_b = rating_b + k_factor * (actual_score_b - expected_score_b)

  return new_rating_a, new_rating_b

def expected_elo_score(rating_a, rating_b):
  """

  Args:
    rating_a: The current ELO rating of player A.
    rating_b: The current ELO rating of player B.

  Returns:
    A tuple containing the expected score for player A and player B.
  """
  expected_score_a = 1 / (1 + 10 ** ((rating_b - rating_a) / 400))
  expected_score_b = 1 / (1 + 10 ** ((rating_a - rating_b) / 400))

  return expected_score_a, expected_score_b

In [ ]:
# prompt: calculate an ELO rating for all the games

# Initialize ELO ratings for each player
elo_ratings = {}
for player in distinct_players:
  if player:
    elo_ratings[player] = 1500  # Starting ELO rating

# Calculate ELO ratings for each game
for index, row in df.iterrows():
  team1_player1 = row['team 1 player 1']
  team1_player2 = row['team 1 player 2']
  team2_player1 = row['team 2 player 1']
  team2_player2 = row['team 2 player 2']
  winner1 = row['winner #1']
  winner2 = row['winner #2']
  loser1 = row['loser #1']
  loser2 = row['loser #2']

  if team1_player1 in elo_ratings and team1_player2 in elo_ratings and team2_player1 in elo_ratings and team2_player2 in elo_ratings:
    # Calculate the average ELO rating for each team
    team1_rating = (elo_ratings[team1_player1] + elo_ratings[team1_player2]) / 2
    team2_rating = (elo_ratings[team2_player1] + elo_ratings[team2_player2]) / 2

    df['team1_rating'] = team1_rating
    df['team2_rating'] = team2_rating

    expected_result_team1, expected_result_team2 = expected_elo_score(team1_rating, team2_rating)
    df['expected_result_team1'] = expected_result_team1
    df['expected_result_team2'] = expected_result_team2

    # Determine the score for each team based on the winners and losers
    if winner1 == team1_player1 or winner1 == team1_player2:
      team1_score = 1
      team2_score = 0
    else:
      team1_score = 0
      team2_score = 1

    # Calculate new ELO ratings for each player
    new_rating_team1_player1, new_rating_team2_player1 = calculate_elo_rating(elo_ratings[team1_player1], elo_ratings[team2_player1], team1_score, team2_score)
    new_rating_team1_player2, new_rating_team2_player2 = calculate_elo_rating(elo_ratings[team1_player2], elo_ratings[team2_player2], team1_score, team2_score)

    df['new_rating_team1_player1'] = new_rating_team1_player1
    df['new_rating_team2_player1'] = new_rating_team2_player1
    df['new_rating_team1_player2'] = new_rating_team1_player2
    df['new_rating_team2_player2'] = new_rating_team2_player2

    # Update ELO ratings for each player
    elo_ratings[team1_player1] = new_rating_team1_player1
    elo_ratings[team1_player2] = new_rating_team1_player2
    elo_ratings[team2_player1] = new_rating_team2_player1
    elo_ratings[team2_player2] = new_rating_team2_player2

# Create a DataFrame from the ELO ratings
df_elo_ratings = pd.DataFrame.from_dict(elo_ratings, orient='index', columns=['ELO Rating'])
df_elo_ratings = df_elo_ratings.reset_index().rename(columns={'index': 'Player'})

# Print the DataFrame with ELO ratings
print(df_elo_ratings.sort_values(by=['ELO Rating'], ascending=False))


In [ ]:
# prompt: export df_elo_ratings back to google sheets and include a date and time

from datetime import datetime

# Get the current date and time
now = datetime.now()
date_time_string = now.strftime("%Y-%m-%d %H:%M:%S")

# Add a new column for the date and time
df_elo_ratings['Date and Time'] = date_time_string

# sort
df_elo_ratings.sort_values(by=['ELO Rating'])

# Convert the DataFrame to a list of lists
elo_ratings_list = df_elo_ratings.sort_values(by=['ELO Rating'], ascending=False).values.tolist()

# Get the worksheet to update
worksheet_elo_ratings = sheet.worksheet("elo_ratings")

# Clear existing data in the worksheet
worksheet_elo_ratings.clear()

# Update the worksheet with the new data
worksheet_elo_ratings.update([df_elo_ratings.sort_values(by=['ELO Rating']).columns.values.tolist()] + elo_ratings_list)


In [ ]:
# prompt: show the elo rating in the df dataframe before and after each game

# Initialize ELO ratings for each player
elo_ratings = {}
for player in distinct_players:
  if player:
    elo_ratings[player] = 1500  # Starting ELO rating

# Calculate ELO ratings for each game
for index, row in df.iterrows():
  team1_player1 = row['team 1 player 1']
  team1_player2 = row['team 1 player 2']
  team2_player1 = row['team 2 player 1']
  team2_player2 = row['team 2 player 2']
  winner1 = row['winner #1']
  winner2 = row['winner #2']
  loser1 = row['loser #1']
  loser2 = row['loser #2']

  if team1_player1 in elo_ratings and team1_player2 in elo_ratings and team2_player1 in elo_ratings and team2_player2 in elo_ratings:
    # Calculate the average ELO rating for each team
    team1_rating_before = (elo_ratings[team1_player1] + elo_ratings[team1_player2]) / 2
    team2_rating_before = (elo_ratings[team2_player1] + elo_ratings[team2_player2]) / 2

    df.loc[index, 'team1_rating_before'] = team1_rating_before
    df.loc[index, 'team2_rating_before'] = team2_rating_before

    expected_result_team1, expected_result_team2 = expected_elo_score(team1_rating_before, team2_rating_before)
    df.loc[index, 'expected_result_team1'] = expected_result_team1
    df.loc[index, 'expected_result_team2'] = expected_result_team2

    # Determine the score for each team based on the winners and losers
    if winner1 == team1_player1 or winner1 == team1_player2:
      team1_score = 1
      team2_score = 0
    else:
      team1_score = 0
      team2_score = 1

    # Calculate new ELO ratings for each player
    new_rating_team1_player1, new_rating_team2_player1 = calculate_elo_rating(elo_ratings[team1_player1], elo_ratings[team2_player1], team1_score, team2_score)
    new_rating_team1_player2, new_rating_team2_player2 = calculate_elo_rating(elo_ratings[team1_player2], elo_ratings[team2_player2], team1_score, team2_score)

    df.loc[index, 'new_rating_team1_player1'] = new_rating_team1_player1
    df.loc[index, 'new_rating_team2_player1'] = new_rating_team2_player1
    df.loc[index, 'new_rating_team1_player2'] = new_rating_team1_player2
    df.loc[index, 'new_rating_team2_player2'] = new_rating_team2_player2

    # Update ELO ratings for each player
    elo_ratings[team1_player1] = new_rating_team1_player1
    elo_ratings[team1_player2] = new_rating_team1_player2
    elo_ratings[team2_player1] = new_rating_team2_player1
    elo_ratings[team2_player2] = new_rating_team2_player2

    # Calculate the average ELO rating for each team after the game
    team1_rating_after = (elo_ratings[team1_player1] + elo_ratings[team1_player2]) / 2
    team2_rating_after = (elo_ratings[team2_player1] + elo_ratings[team2_player2]) / 2

    df.loc[index, 'team1_rating_after'] = team1_rating_after
    df.loc[index, 'team2_rating_after'] = team2_rating_after

# Print the DataFrame with ELO ratings before and after each game
print(df)


In [ ]:
# prompt: show games from df only with 'Kyle' and create a new column showing Kyle's rating over time

# Filter games with 'Kyle'
df_kyle = df[
    df['team 1 player 1'].str.contains('Kyle') |
    df['team 1 player 2'].str.contains('Kyle') |
    df['team 2 player 1'].str.contains('Kyle') |
    df['team 2 player 2'].str.contains('Kyle')
]

# Function to get Kyle's rating for a given row
def get_kyle_rating(row):
  if row['team 1 player 1'] == 'Kyle':
    return row['new_rating_team1_player1']
  elif row['team 1 player 2'] == 'Kyle':
    return row['new_rating_team1_player2']
  elif row['team 2 player 1'] == 'Kyle':
    return row['new_rating_team2_player1']
  elif row['team 2 player 2'] == 'Kyle':
    return row['new_rating_team2_player2']
  else:
    return None

# Apply the function to create a new column
df_kyle['Kyle Rating'] = df_kyle.apply(get_kyle_rating, axis=1)

# Print the filtered DataFrame
print(df_kyle)


In [ ]:
# prompt: Add a column for Kyle's game number

# Create a new column for Kyle's game number
df_kyle['Kyle Game Number'] = range(1, len(df_kyle) + 1)

# Print the updated DataFrame
print(df_kyle)


In [ ]:
# prompt: graph 'Kyle Rating' over time

import matplotlib.pyplot as plt

# Sort the DataFrame by the 'Game #' column
df_kyle = df_kyle.sort_values(by=['Kyle Game Number'])

# Plot the 'Kyle Rating' over time
plt.plot(df_kyle['Kyle Game Number'], df_kyle['Kyle Rating'])
plt.xlabel('Kyle Game Number')
plt.ylabel('Kyle Rating')
plt.title('Kyle\'s ELO Rating Over Time')
plt.show()


In [ ]:
# prompt: create an interactive graph showing each player's rating over time

import pandas as pd
import plotly.express as px

# Create a list to store the player ratings over time
player_ratings_over_time = []

# Iterate through the DataFrame
for index, row in df.iterrows():
  # Get the players and their new ratings
  players = [
      {'player': row['team 1 player 1'], 'rating': row['new_rating_team1_player1']},
      {'player': row['team 1 player 2'], 'rating': row['new_rating_team1_player2']},
      {'player': row['team 2 player 1'], 'rating': row['new_rating_team2_player1']},
      {'player': row['team 2 player 2'], 'rating': row['new_rating_team2_player2']}
  ]
  # Add the player ratings to the list
  for player in players:
    if player['player']:
      player_ratings_over_time.append({
          'Game': index + 1,
          'Player': player['player'],
          'Rating': player['rating']
      })

# Create a DataFrame from the player ratings over time
df_ratings_over_time = pd.DataFrame(player_ratings_over_time)

# Create an interactive line plot using Plotly
fig = px.line(df_ratings_over_time, x='Game', y='Rating', color='Player', title='Player Ratings Over Time')
fig.show()


In [ ]:
# prompt: create an interactive graph showing each player's rating over time based on only the games they've played in

import pandas as pd
# Create a list to store the player ratings over time
player_ratings_over_time = []

# Iterate through the DataFrame
for index, row in df.iterrows():
  # Get the players and their new ratings
  players = [
      {'player': row['team 1 player 1'], 'rating': row['new_rating_team1_player1']},
      {'player': row['team 1 player 2'], 'rating': row['new_rating_team1_player2']},
      {'player': row['team 2 player 1'], 'rating': row['new_rating_team2_player1']},
      {'player': row['team 2 player 2'], 'rating': row['new_rating_team2_player2']}
  ]
  # Add the player ratings to the list
  for player in players:
    if player['player']:
      player_ratings_over_time.append({
          'Game': index + 1,
          'Player': player['player'],
          'Rating': player['rating']
      })

# Create a DataFrame from the player ratings over time
df_ratings_over_time = pd.DataFrame(player_ratings_over_time)

# Filter out games where the player was not present
df_ratings_over_time = df_ratings_over_time.sort_values(by=['Player', 'Game'])
df_ratings_over_time['Game Number'] = df_ratings_over_time.groupby('Player').cumcount() + 1

# Create an interactive line plot using Plotly
fig = px.line(df_ratings_over_time, x='Game Number', y='Rating', color='Player', title='Player Ratings Over Time (Games Played)')
fig.show()


In [ ]:
# prompt: get all unique values in Date column and sort

# Get all unique values in the 'Date' column
unique_dates = df['Date'].unique()

# Sort the unique dates
unique_dates.sort()

# Print the sorted unique dates
print(unique_dates)


In [ ]:
# prompt: create an interactive graph showing each player's rating over time by month and year with x axis starting on the first date and ending on the last date

import pandas as pd
# Convert the 'Date' column to datetime objects
df['Date'] = pd.to_datetime(df['Date'])

# Create a list to store the player ratings over time
player_ratings_over_time = []

# Iterate through the DataFrame
for index, row in df.iterrows():
  # Get the players and their new ratings
  players = [
      {'player': row['team 1 player 1'], 'rating': row['new_rating_team1_player1'], 'date': row['Date']},
      {'player': row['team 1 player 2'], 'rating': row['new_rating_team1_player2'], 'date': row['Date']},
      {'player': row['team 2 player 1'], 'rating': row['new_rating_team2_player1'], 'date': row['Date']},
      {'player': row['team 2 player 2'], 'rating': row['new_rating_team2_player2'], 'date': row['Date']}
  ]
  # Add the player ratings to the list
  for player in players:
    if player['player']:
      player_ratings_over_time.append({
          'Date': player['date'],
          'Player': player['player'],
          'Rating': player['rating']
      })

# Create a DataFrame from the player ratings over time
df_ratings_over_time = pd.DataFrame(player_ratings_over_time)

# Sort the DataFrame by date and player
df_ratings_over_time = df_ratings_over_time.sort_values(by=['Date', 'Player'])

# Create an interactive line plot using Plotly
fig = px.line(df_ratings_over_time, x='Date', y='Rating', color='Player', title='Player Ratings Over Time')
fig.show()


In [ ]:
# prompt: add a record with today's date and the player's latest ranking to df_ratings_over_time

import pandas as pd
from datetime import date

# Get today's date
today = date.today()

# Get the latest ratings for each player
latest_ratings = df_elo_ratings.set_index('Player')['ELO Rating'].to_dict()

# Create a list to store the new records
new_records = []

# Add a new record for each player with today's date and their latest rating
for player, rating in latest_ratings.items():
  new_records.append({
      'Date': today,
      'Player': player,
      'Rating': rating
  })

# Create a DataFrame from the new records
new_df = pd.DataFrame(new_records)

# Append the new records to the existing DataFrame
df_ratings_over_time = pd.concat([df_ratings_over_time, new_df], ignore_index=True)
df_ratings_over_time

In [ ]:
# Create an interactive line plot using Plotly
fig = px.line(df_ratings_over_time, x='Date', y='Rating', color='Player', title='Player Ratings Over Time')
fig.show()

In [ ]:
# prompt: highlight Kyle and Mark in the plot

# Create an interactive line plot using Plotly
fig = px.line(df_ratings_over_time, x='Date', y='Rating', color='Player', title='Player Ratings Over Time')

# Highlight Kyle and Mark
fig.update_traces(line=dict(width=10), selector=dict(name='Kyle'))
fig.update_traces(line=dict(width=10), selector=dict(name='Mark'))
fig.update_traces(line=dict(width=10), selector=dict(name='Kevin'))

fig.show()


In [ ]:
# prompt: select only Kyle in the plot

# Create an interactive line plot using Plotly
fig = px.line(df_ratings_over_time, x='Date', y='Rating', color='Player', title='Player Ratings Over Time')

# Highlight Kyle
# Set default selection
fig.update_traces(visible="legendonly") #<----- deselect all lines
fig.data[1].visible=True   #<------ display the second player

fig.show()


In [ ]:
# selectedpoints=[True if p == 'Mark' else False for p in df_ratings_over_time['Player']]
# selectedpoints

In [ ]:
# prompt: df_ratings_over_time['Player'] where player is Kyle

df_ratings_over_time[df_ratings_over_time['Player'] == 'Kyle']


In [ ]:
fig.data[1]

In [ ]:
# prompt: obtain the legendgroup from fig.data[1]

legendgroup = fig.data[1].legendgroup
legendgroup

In [ ]:
# prompt: get the number of legendgroups

length_fig_data = len(fig.data)


In [ ]:
# prompt: instead of data[0] show code using where value = 'Kyle'

# Create an interactive line plot using Plotly
fig = px.line(df_ratings_over_time, x='Date', y='Rating', color='Player', title='Player Ratings Over Time')

# Highlight Kyle
# Set default selection
fig.update_traces(visible="legendonly") #<----- deselect all lines

# Select Kyle's data
# fig.update_traces(selectedpoints=[True if p == 'Kyle' else False for p in df_ratings_over_time['Player']], selector=dict(type='scatter'))
# fig.update_traces(selectedpoints=[True if p == 'Kyle' else False for p in df_ratings_over_time['Player']])
# fig.update_traces(selectedpoints=[True if p == 'Mark' else False for p in df_ratings_over_time['Player']], selector=dict(type='scatter'))
# for k in df_ratings_over_time[df_ratings_over_time['Player'] == 'Kyle']:
#      fig.update_traces(visible=True, selector=k)
# fig.update_traces(visible=True, selector=df_ratings_over_time[df_ratings_over_time['Player'] == 'Kyle'])
length_fig_data = len(fig.data)
for k in range(0, length_fig_data):
  if fig.data[k].legendgroup == 'Kyle':
    fig.data[k].visible = True

fig.update_xaxes(range=[df['Date'].min(), df['Date'].max()])  # Replace with your desired date range
fig.update_yaxes(range=[df_ratings_over_time['Rating'].min(), df_ratings_over_time['Rating'].max()])  # Replace with your desired date range

fig.show()

In [ ]:
# prompt: find the minimum date and the maximum date

min_date = df['Date'].min()
max_date = df['Date'].max()

print(f"Minimum date: {min_date}")
print(f"Maximum date: {max_date}")


In [ ]:
# prompt: show the current folder path

!pwd


In [ ]:
# prompt: export df to an excel file

df.to_excel('output.xlsx', index=False)


In [ ]:
# prompt: generate a dataframe showing winning percentage for each player against all other players

import pandas as pd
# Create a dictionary to store the head-to-head win percentages
head_to_head_win_percentages = {}

# Iterate over each player
for player in distinct_players:
  if player:
    head_to_head_win_percentages[player] = {}
    for opponent in distinct_players:
      if opponent:
        if player != opponent:
          # Initialize win and loss counts
          wins = 0
          losses = 0
          # Iterate through the DataFrame to find matches between the player and opponent
          for index, row in df.iterrows():
            if (row['team 1 player 1'] == player and row['team 2 player 1'] == opponent) or \
               (row['team 1 player 1'] == player and row['team 2 player 2'] == opponent) or \
               (row['team 1 player 2'] == player and row['team 2 player 1'] == opponent) or \
               (row['team 1 player 2'] == player and row['team 2 player 2'] == opponent) or \
               (row['team 2 player 1'] == player and row['team 1 player 1'] == opponent) or \
               (row['team 2 player 1'] == player and row['team 1 player 2'] == opponent) or \
               (row['team 2 player 2'] == player and row['team 1 player 1'] == opponent) or \
               (row['team 2 player 2'] == player and row['team 1 player 2'] == opponent):
              if row['winner #1'] == player or row['winner #2'] == player:
                wins += 1
              else:
                losses += 1
          # Calculate the win percentage
          if wins + losses > 0:
            win_percentage = (wins / (wins + losses)) * 100
          else:
            win_percentage = 0
          # Store the win percentage in the dictionary
          head_to_head_win_percentages[player][opponent] = win_percentage
        else:
          # If player and opponent are the same, set win percentage to 0
          head_to_head_win_percentages[player][opponent] = 0

# Create a DataFrame from the head-to-head win percentages dictionary
df_head_to_head = pd.DataFrame.from_dict(head_to_head_win_percentages)

# Print the DataFrame
print(df_head_to_head)


In [ ]:
# prompt: tranpose df_head_to_head
# to get winning player on left, losing player on top

# Transpose the DataFrame
df_head_to_head_transposed = df_head_to_head.transpose()

# Print the transposed DataFrame
print(df_head_to_head_transposed)


In [ ]:
# !pip install glicko2

In [ ]:
# prompt: instead of an ELO rating use a glicko rating

from google.colab import auth
import gspread
from google.auth import default
import pandas as pd
import math
import matplotlib.pyplot as plt
import plotly.express as px
from datetime import date
# from glicko2 import Rating
import glicko2

# Initialize Glicko ratings for each player
glicko_ratings = {}
for player in distinct_players:
  if player:
    glicko_ratings[player] = glicko2.Rating()  # Starting Glicko rating

# Calculate Glicko ratings for each game
for index, row in df.iterrows():
  team1_player1 = row['team 1 player 1']
  team1_player2 = row['team 1 player 2']
  team2_player1 = row['team 2 player 1']
  team2_player2 = row['team 2 player 2']
  winner1 = row['winner #1']
  winner2 = row['winner #2']
  loser1 = row['loser #1']
  loser2 = row['loser #2']

  if team1_player1 in glicko_ratings and team1_player2 in glicko_ratings and team2_player1 in glicko_ratings and team2_player2 in glicko_ratings:
    # Determine the score for each team based on the winners and losers
    if winner1 == team1_player1 or winner1 == team1_player2:
      team1_score = 1
      team2_score = 0
    else:
      team1_score = 0
      team2_score = 1

    # Calculate new Glicko ratings for each player
    new_rating_team1_player1, new_rating_team2_player1 = calculate_glicko_rating(glicko_ratings[team1_player1], glicko_ratings[team2_player1], team1_score, team2_score)
    new_rating_team1_player2, new_rating_team2_player2 = calculate_glicko_rating(glicko_ratings[team1_player2], glicko_ratings[team2_player2], team1_score, team2_score)

    df.loc[index, 'new_rating_team1_player1'] = new_rating_team1_player1.mu
    df.loc[index, 'new_rating_team2_player1'] = new_rating_team2_player1.mu
    df.loc[index, 'new_rating_team1_player2'] = new_rating_team1_player2.mu
    df.loc[index, 'new_rating_team2_player2'] = new_rating_team2_player2.mu

    # Update Glicko ratings for each player
    glicko_ratings[team1_player1] = new_rating_team1_player1
    glicko_ratings[team1_player2] = new_rating_team1_player2
    glicko_ratings[team2_player1] = new_rating_team2_player1
    glicko_ratings[team2_player2] = new_rating_team2_player2

# Create a DataFrame from the Glicko ratings
df_glicko_ratings = pd.DataFrame.from_dict({player: rating.mu for player, rating in glicko_ratings.items()}, orient='index', columns=['Glicko Rating'])
df_glicko_ratings = df_glicko_ratings.reset_index().rename(columns={'index': 'Player'})

# Print the DataFrame with Glicko ratings
print(df_glicko_ratings.sort_values(by=['Glicko Rating'], ascending=False))

# Function to calculate Glicko ratings
def calculate_glicko_rating(rating1, rating2, score1, score2):
  """
  Calculates new Glicko ratings for two players based on the game result.

  Args:
    rating1: Glicko rating object for player 1.
    rating2: Glicko rating object for player 2.
    score1: Score for player 1 (1 for win, 0 for loss).
    score2: Score for player 2 (1 for win, 0 for loss).

  Returns:
    tuple: New Glicko rating objects for player 1 and player 2.
  """
  # Create a list of ratings for the players
  ratings = [rating1, rating2]
  # Create a list of scores
  scores = [score1, score2]
  # Update the ratings based on the scores
  for i in range(2):
    opponent_ratings = [ratings[j] for j in range(2) if j != i]
    opponent_scores = [scores[j] for j in range(2) if j != i]
    ratings[i].update(opponent_ratings, opponent_scores)
  # Return the new ratings
  return ratings[0], ratings[1]
